# Ceneo Scraper
**Struktura opinii w serwisie ceneo.pl:**
|Składowa|Selektor|Zmienna|
|--------|--------|--------|
|*identyfikator opinii*|["data-entry-id"]|`opinion_id`|
|*autora*|span.user-post__author-name|`author`|
|*rekomendację*|span.user-post__author-recomendation > em|`recommend`|
|*liczbę gwiazdek*|span.user-post__score-count|`stars`|
|*treść opinii*|div.user-post__text|`content`|
|*listę wad*|div.review-feature__title review-feature__title--negatives ~ div.review-feature__item|`cons`|
|*listę zalet*|div.review-feature__title review-feature__title--positives ~ div.review-feature__item|`pros`|
|*data wystawienia opinii*|span.user-post__published > time:nth-child(1)["datetime"]|`opinion_date`|
|*data zakupu produktu*|span.user-post__published > time:nth-child(2)["datetime"]|`purchase_date`|
|*ile osób uznało opinię za przydatną*|button.vote-yes["data-total-vote"]|`upvote`|
|*ile osób uznało opinię za nieprzydatną*|button.vote-no["data-total-vote"]|`downvote`|

### 1. Import bibliotek

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import os

In [2]:
selectors = {
    'opinion_id': (None, "data-entry-id"),
    'author': ("span.user-post__author-name",),
    'recommend': ("span.user-post__author-recomendation > em",),
    'stars': ("span.user-post__score-count",),
    'content': ("div.user-post__text",),
    'cons': ("div.review-feature__title--negatives ~ div.review-feature__item", None, True),
    'pros': ("div.review-feature__title--positives ~ div.review-feature__item", None, True),
    'opinion_date': ("span.user-post__published > time:nth-child(1)", "datetime"),
    'purchase_date': ("span.user-post__published > time:nth-child(2)", "datetime"),
    'upvote': ("button.vote-yes", "data-total-vote"),
    'downvote': ("button.vote-no", "data-total-vote")
}

### Do wydobycia danych 

In [3]:
def extract(ancestor, selector=None, attribute=None, return_list=False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select(selector)]
        return [tag.text.strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip()

### 2. Przypisanie adresu url do zmiennej

In [4]:
#product_id = "151210226"
product_id = input("Podaj kod produktu: ")
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"

### 3. Pobranie wszystkich opinii o wskazanym produkcie

In [5]:
all_opinions = []
while(url):
    response = requests.get(url)
    content = response.text
    page = BeautifulSoup(content, "html.parser")
    opinions = page.select("div.js_product-review")
    for opinion in opinions:
        single_opinion = {
            key: extract(opinion, *value)
                for key, value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:
        url = "https://ceneo.pl" + extract(page, "a.pagination__next", "href")
    except TypeError:
        url = None

### 4. Zapis opinii o produkcie do pliku JSON

In [6]:
if not os.path.exists('opinions'):
    os.mkdir('opinions')

with open(f'opinions/{product_id}.json', 'w', encoding="UTF-8") as file:
    json.dump(all_opinions, file, indent=4, ensure_ascii=False)